# Import Libraries & Data

In [1]:
#pip install numpy
#pip install pandas

In [2]:
import os
import glob
import re
import numpy as np
import pandas as pd

### Load and connect all csv files

#### Info on raw data

1. Stella:
- lines before filter: 284.882 --> 259.603     pososto: 259603/284882 = 0.9112
- columns: 714

2. Maria:
- lines before filter: 347.428 --> 313.755     pososto: 313755/347428 = 0.9030
- columns: 714

To Do!!!


1. create label
2. choose if we want all features or selected features
3. σε τι dimension θα τρεξουμε : εχουμε 2 choices λοιπον --> αυτα που ειναι στα slides

--> μεσο Intensity ενος action unit σε εναν x-xrono - ο χρονος ειναι πολυ σημαντικος


!!!to check: preprocessing --> training protocols female klp

In [3]:
# file paths

openface_outputs = '/home/maria/Desktop/Deception_project/openface csv output/'

In [4]:
%time

# load all csv openface outputs (from DOLOS)

openface_files = glob.glob(os.path.join(openface_outputs, '*.csv'))
df_list = [pd.read_csv(file) for file in openface_files]

# concatenate the list into a single df

#df_combined = pd.concat(df_list, ignore_index = True)
#df_combined = [pd.read_csv(f).assign(file_name = os.path.basename(f)) for f in openface_files]

df_combined = pd.concat(
    [pd.read_csv(f).assign(file_name = os.path.basename(f)) for f in openface_files], 
    ignore_index = True
)

print(f"Combined {len(openface_files)} files into {df_combined.shape[0]} total rows.")

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 6.68 μs
Combined 1352 files into 347428 total rows.


In [5]:
df_combined.head()

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name
0,1,0,0.00,0.88,1,-0.266795,0.265201,-0.926547,-0.384724,0.253047,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv
1,2,0,0.04,0.88,1,-0.298252,0.246420,-0.922129,-0.410799,0.239001,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv
2,3,0,0.08,0.88,1,-0.300153,0.241749,-0.922749,-0.413097,0.234598,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv
3,4,0,0.12,0.88,1,-0.300693,0.241151,-0.922730,-0.412985,0.233968,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv
4,5,0,0.16,0.98,1,0.276003,0.192164,-0.941751,0.125905,0.171836,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv


# Preparation of Data

In [6]:
#df_combined.columns

# strip whitespace from column names 
df_combined.columns = df_combined.columns.str.strip()

In [7]:
# create label column from file name

conditions = [
    df_combined['file_name'].str.contains('lie', case = False) | 
    df_combined['file_name'].str.contains('deception', case = False),
    
    df_combined['file_name'].str.contains('truth', case = False) |
    df_combined['file_name'].str.contains('true', case = False)
]

choices = ['lie', 'truth']

df_combined['label'] = np.select(conditions, choices, default = pd.NA)

df_combined

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label
0,1,0,0.00,0.88,1,-0.266795,0.265201,-0.926547,-0.384724,0.253047,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
1,2,0,0.04,0.88,1,-0.298252,0.246420,-0.922129,-0.410799,0.239001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
2,3,0,0.08,0.88,1,-0.300153,0.241749,-0.922749,-0.413097,0.234598,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
3,4,0,0.12,0.88,1,-0.300693,0.241151,-0.922730,-0.412985,0.233968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
4,5,0,0.16,0.98,1,0.276003,0.192164,-0.941751,0.125905,0.171836,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347423,146,0,5.80,0.93,1,0.220692,-0.050918,-0.974014,0.050902,-0.056871,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth
347424,147,0,5.84,0.98,1,0.233077,-0.042794,-0.971516,0.053544,-0.046702,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth
347425,148,0,5.88,0.98,1,0.235445,-0.043778,-0.970901,0.054714,-0.047773,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth
347426,149,0,5.92,0.98,1,0.236376,-0.041068,-0.970793,0.054574,-0.044504,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth


In [8]:
# check that no labels are empty

check_na_label = df_combined[df_combined['label'].isna()]
check_na_label

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label


In [9]:
df_combined

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label
0,1,0,0.00,0.88,1,-0.266795,0.265201,-0.926547,-0.384724,0.253047,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
1,2,0,0.04,0.88,1,-0.298252,0.246420,-0.922129,-0.410799,0.239001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
2,3,0,0.08,0.88,1,-0.300153,0.241749,-0.922749,-0.413097,0.234598,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
3,4,0,0.12,0.88,1,-0.300693,0.241151,-0.922730,-0.412985,0.233968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
4,5,0,0.16,0.98,1,0.276003,0.192164,-0.941751,0.125905,0.171836,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347423,146,0,5.80,0.93,1,0.220692,-0.050918,-0.974014,0.050902,-0.056871,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth
347424,147,0,5.84,0.98,1,0.233077,-0.042794,-0.971516,0.053544,-0.046702,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth
347425,148,0,5.88,0.98,1,0.235445,-0.043778,-0.970901,0.054714,-0.047773,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth
347426,149,0,5.92,0.98,1,0.236376,-0.041068,-0.970793,0.054574,-0.044504,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth


In [10]:
# keep only lines where confidence >= 0.8 & success == 1

dolos_dataset = df_combined[(df_combined['confidence'] >= 0.8) & (df_combined['success'] == 1)]
dolos_dataset

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label
0,1,0,0.00,0.88,1,-0.266795,0.265201,-0.926547,-0.384724,0.253047,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
1,2,0,0.04,0.88,1,-0.298252,0.246420,-0.922129,-0.410799,0.239001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
2,3,0,0.08,0.88,1,-0.300153,0.241749,-0.922749,-0.413097,0.234598,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
3,4,0,0.12,0.88,1,-0.300693,0.241151,-0.922730,-0.412985,0.233968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
4,5,0,0.16,0.98,1,0.276003,0.192164,-0.941751,0.125905,0.171836,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347423,146,0,5.80,0.93,1,0.220692,-0.050918,-0.974014,0.050902,-0.056871,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth
347424,147,0,5.84,0.98,1,0.233077,-0.042794,-0.971516,0.053544,-0.046702,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth
347425,148,0,5.88,0.98,1,0.235445,-0.043778,-0.970901,0.054714,-0.047773,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth
347426,149,0,5.92,0.98,1,0.236376,-0.041068,-0.970793,0.054574,-0.044504,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth


In [ ]:
# drop duplicates id any

duplicates = dolos_dataset.duplicated()
dolos_dataset[duplicates]

# Export Final Dataset

In [12]:
# export to pickle in order to not append all CSVs every time

df_combined.to_pickle('df_combined.pkl')

In [13]:
dolos_dataset.to_pickle('dolos_openface_merged.pkl')
dolos_dataset.to_csv('dolos_openface_merged.csv', index = False)